In [2]:
# Install required libraries
!pip install dash pandas plotly

# Import libraries
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px
from dash.dependencies import Input, Output
from google.colab import drive

# Load Dataset

file_path = "/content/sample_data/amazon.csv"
df = pd.read_csv(file_path)

# Data Cleaning
df = df.dropna()
df['actual_price'] = df['actual_price'].astype(str).str.replace(r'[₹,]', '', regex=True).astype(float)
df['discounted_price'] = df['discounted_price'].astype(str).str.replace(r'[₹,]', '', regex=True).astype(float)
df['discount_percentage'] = df['discount_percentage'].astype(str).str.replace('%', '', regex=True).astype(float)

# Initialize Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Amazon Sales Dashboard", style={'text-align': 'center'}),

    dcc.Dropdown(
        id="category_dropdown",
        options=[{'label': cat, 'value': cat} for cat in df['category'].unique()],
        multi=False,
        placeholder="Select a Category",
    ),

    dcc.Graph(id="sales_by_category"),
    dcc.Graph(id="top_discounted_products"),
    dcc.Graph(id="ratings_distribution")
])

# Callback function to update graphs dynamically
@app.callback(
    [Output("sales_by_category", "figure"),
     Output("top_discounted_products", "figure"),
     Output("ratings_distribution", "figure")],
    [Input("category_dropdown", "value")]
)
def update_graphs(selected_category):
    filtered_df = df if selected_category is None else df[df["category"] == selected_category]

    # Sales by Category
    sales_fig = px.bar(filtered_df.groupby("category")["discounted_price"].sum().reset_index(),
                       x="category", y="discounted_price", title="Total Sales by Category", color="category")

    # Top Discounted Products
    top_discount_fig = px.bar(filtered_df.nlargest(10, "discount_percentage"),
                              x="product_name", y="discount_percentage", title="Top Discounted Products", color="discount_percentage")

    # Ratings Distribution
    rating_fig = px.histogram(filtered_df, x="rating", title="Ratings Distribution", nbins=20, color="rating")

    return sales_fig, top_discount_fig, rating_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=False)


<IPython.core.display.Javascript object>